# Описание

## Цель

Кластеризация + Класификация + NLP

## Задачи

1. Предложите решение по выделению подмножества тематик и их описания из
общего корпуса цитат.
2. Предположим, Йеннифер и Трисс поставят эту активность как регулярную, проводя
интервью раз в месяц по разным доменам предприятия. Предложите варианты, как
Лютику сделать для них легковесную систему по обработке новых цитат из опросов с
учетом уже накопленной истории извлеченных тематик и тегов.

# Установка

In [59]:
pip install nltk

In [60]:
pip install emoji

In [61]:
pip install num2words

In [62]:
pip install pymorphy2

In [107]:
pip install razdel

In [63]:
pip install pandas gensim nltk pyLDAvis

In [115]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.3 MB/s eta 0:00:00


## Импорт библиотек

In [120]:
import re

import numpy as np
import pandas as pd
import json
from google.colab import drive
import warnings

from num2words import num2words

import nltk

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from gensim import corpora
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel

import string
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import pymorphy2
import razdel

import optuna

## Настройка

In [65]:
#drive.mount('/content/drive')

In [66]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [67]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Чтение

In [68]:
data = pd.read_json(r"/content/drive/MyDrive/case gpn/case oil retail/cintra_phoenix_oils_hr_mgck_feather.json")
data

,id,quote
0,0,«У ннас среди ночi в райооне 55 часов упала по...
1,1,Программы повышения квалификации через гильдию...
2,2,"""Мурр... Новиград - город контрастов, да. Но в..."
3,3,"Типа, вот уже полгода, как мы ждем установки э..."
4,4,"""Теперь всьё так просто! Раньше бумажки летали..."
...,...,...
954,954,"«ВВ го Rhode, я не знаю,, у них нет тут ночн..."
955,955,"Праздники у нас как-то так, ну, были и были, н..."
956,956,“В жеенском кооолектииивъ всё раавно скллокк ...
957,957,"Потом ещё нехватa у нас очень большая, у нас с..."


# Обработка текстовых данных

## Преобразование всех букв в строчные

In [69]:
def to_lowercase(text):
  return text.lower()

data['quote'] = data['quote'].apply(to_lowercase)

## Замена английских букв русскими буквами

In [70]:
def replace_eng_with_rus(text):
  eng_to_rus = {
    'a': 'а',
    'b': 'б',
    'd': 'д',
    'c': 'с',
    'e': 'е',
    'h': 'н',
    'i': 'и',
    'k': 'к',
    'm': 'м',
    'o': 'о',
    'p': 'р',
    'r': 'р',
    't': 'т',
    'u': 'у',
    'y': 'у',
    'x': 'х',
    'z': 'з',
    'j': 'й'
    }
  for eng, rus in eng_to_rus.items():
    text = text.replace(eng, rus)
  return text

data['quote'] = data['quote'].apply(replace_eng_with_rus)

## Локальные замены чисел символов русскими буквами

In [71]:
def replace_num_with_letter_in_words(text):
  num_to_letter = {
    '3': 'е',
    '4': 'д',
    '0': 'о',
    '@': 'а',
    'є': 'е',
    'é': 'е',
    'j': 'й',
    'ó': 'о'
    }
  for num, letter in num_to_letter.items():
    text = re.sub(r'\b' + num + r'(\D)', r'' + letter + r'\1', text)
    text = re.sub(r'(\D)' + num + r'\b', r'\1' + letter, text)
    text = re.sub(r'(\D)' + num + r'(\D)', r'\1' + letter + r'\2', text)
  return text


data['quote'] = data['quote'].apply(replace_num_with_letter_in_words)

## Замена эмоджи

In [72]:
def replace_emoji_to_rus(text):
  emoji_to_rus_dict = {
        '😀': 'улыбка',
        '😂': 'смех',
        '😊': 'улыбка',
        '😍': 'влюбленность',
        '😎': 'круто',
        '😢': 'грусть',
        '😡': 'злость',
        '💔': 'сердце',
        '💖': 'сердце',
        '💪': 'сила',
        '👍': 'лайк',
        '👎': 'дизлайк',
        '🙏': 'молитва',
        '🤔': 'размышление',
        '🤗': 'обнимашки',
        '🤩': 'восхищение',
        '🤯': 'взрыв мозга',
        '🤢': 'тошнота',
        '🤮': 'рвота',
        '🤡': 'клоун',
        '🤠': 'ковбой',
        '🤑': 'деньги',
        '🤓': 'ботан',
        '🤥': 'лгун',
        '🤧': 'чихание',
        '🎉': 'победа',
        '🔥': 'огонь',
        '🤬': 'ругань',
        '💣': 'бомба'}

  for emoji_char, rus_word in emoji_to_rus_dict.items():
    text = text.replace(emoji_char, rus_word)
  return text

data['quote'] = data['quote'].apply(replace_emoji_to_rus)

## Замена аббревиатур

In [73]:
def replace_abbreviations(text):
  abbreviations_dict = {
      'суоч': 'система управления операционными часами',
      'змс': 'зачарованная маслостанция',
      'нсс': 'назаирская система слив',
      'лдс': 'лекарский договор страхования',
      'цист': 'цинтрийский стандарт',
      'сокг': 'система оплаты карточкой гильдии',
      'цмф': 'цинтрийские масла феникса',
      'цммф': 'цинтрийские масла феникса',
  }

  for abbreviation, expansion in abbreviations_dict.items():
    text = text.replace(abbreviation, expansion)

  return text

data['quote'] = data['quote'].apply(replace_abbreviations)

## Замена чисел

In [74]:
def replace_numbers_with_words(text):
  def number_to_words(match):
    number = int(match.group(0))
    return num2words(number, lang='ru')
  text = re.sub(r'\b\d+\b', number_to_words, text)
  return text

data['quote'] = data['quote'].apply(replace_numbers_with_words)

## Удаление символов

In [75]:
def clean_text(text):
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'\d+', ' ', text)
  text = re.sub(r'(\w)\1+', r'\1', text)
  text = ' '.join(word for word in text.split() if len(word) > 1)
  text = re.sub(r'\s+', ' ', text)
  return text.strip()

data['quote'] = data['quote'].apply(clean_text)

In [76]:
data

,id,quote
0,0,нас среди ночи районе пятьдесят пять часов упа...
1,1,програмы повышения квалификаци через гильдию ц...
2,2,мур новиград город контрастов да но цинтрийски...
3,3,типа вот уже полгода как мы ждем установки эти...
4,4,теперь всьё так просто раньше бумажки летали т...
...,...,...
954,954,го рноде не знаю них нет тут ночных смен помое...
955,955,праздники нас както так ну были были но вот ка...
956,956,женском колективъ всё равно склок не избежать
957,957,потом ещё нехвата нас очень большая нас стоит ...


# LDA

## Леммитизация

### Слова, которые не должны отбрасываться

In [96]:
include_words = {'система', 'зачарованная', 'маслостанция', 'назаирская', 'слив', 'лекарский', 'цинтрийский', 'гильдии', 'цинтрийские', 'феникса', 'масла'}

### Слова, которые должны быть исключены обязательно

In [97]:
exclude_words = {'хотя', 'быть', 'это', 'всё', 'все', 'тип', 'мы', 'скатя', 'весь', 'очень', 'который', 'наш', 'очень', 'както', 'тип', 'мы', 'бывать', 'этот', 'чтотъ', 'сто', 'такой'}

In [110]:
stop_words = set(stopwords.words('russian'))
morph = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    tokens = [token.text for token in razdel.tokenize(text.lower())]
    tokens = [token for token in tokens if token.isalpha() and (token not in stop_words or token in include_words)]
    tokens = [token for token in tokens if token not in exclude_words]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

data['tokens'] = data['quote'].apply(preprocess_text)

In [111]:
data

,id,quote,tokens
0,0,нас среди ночи районе пятьдесят пять часов упа...,"[среди, ночь, район, пятьдесят, пять, час, упа..."
1,1,програмы повышения квалификаци через гильдию ц...,"[програм, повышение, квалификаци, гильдия, цин..."
2,2,мур новиград город контрастов да но цинтрийски...,"[мур, новиград, город, контраст, цинтрийский, ..."
3,3,типа вот уже полгода как мы ждем установки эти...,"[тип, полгода, ждать, установка, этот, фильтр,..."
4,4,теперь всьё так просто раньше бумажки летали т...,"[всьё, просто, ранний, бумажка, летать, терять..."
...,...,...,...
954,954,го рноде не знаю них нет тут ночных смен помое...,"[го, рнода, знать, ночной, смена, помоему, два..."
955,955,праздники нас както так ну были были но вот ка...,"[праздник, организовать, сначала, вроде, восто..."
956,956,женском колективъ всё равно склок не избежать,"[женский, колективъ, равно, склока, избежать]"
957,957,потом ещё нехвата нас очень большая нас стоит ...,"[ещё, нехват, больший, стоить, наш, помещение,..."


## Идентификация текста

In [112]:
dictionary = corpora.Dictionary(data['tokens'])
corpus = [dictionary.doc2bow(tokens) for tokens in data['tokens']]

## LDA анализ + оптимизация

In [123]:
def objective(trial):
    num_topics = trial.suggest_int('num_topics', 2, 10)
    alpha = trial.suggest_loguniform('alpha', 1e-3, 1e0)
    eta = trial.suggest_loguniform('eta', 1e-3, 1e0)
    passes = trial.suggest_int('passes', 5, 20)
    chunksize = trial.suggest_int('chunksize', 50, 200)
    decay = trial.suggest_loguniform('decay', 1e-3, 1e0)

    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=num_topics,
        alpha=alpha,
        eta=eta,
        passes=passes,
        chunksize=chunksize,
        decay=decay
    )

    coherence_model = CoherenceModel(model=lda_model, texts=data['tokens'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    return coherence_score

In [124]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)
best_params = study.best_params
best_params

[I 2024-11-07 16:48:02,532] A new study created in memory with name: no-name-4f02f164-3c00-4f5f-8388-1ea45be673db
<ipython-input-123-7ebc06251c1d>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  alpha = trial.suggest_loguniform('alpha', 1e-3, 1e0)
<ipython-input-123-7ebc06251c1d>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  eta = trial.suggest_loguniform('eta', 1e-3, 1e0)
<ipython-input-123-7ebc06251c1d>:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  decay = trial.suggest_loguniform('decay', 1e-3, 1e0)
[I

{'num_topics': 4,
 'alpha': 0.24509055680622455,
 'eta': 0.4779667811102778,
 'passes': 17,
 'chunksize': 198,
 'decay': 0.08256483881276765}

In [125]:
best_lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=best_params['num_topics'],
    alpha=best_params['alpha'],
    eta=best_params['eta'],
    passes=best_params['passes'],
    chunksize=best_params['chunksize'],
    decay=best_params['decay']
)

In [126]:
for idx, topic in best_lda_model.print_topics(-1):
    print(f"Тема {idx}: {topic}")

Тема 0: 0.010*"человек" + 0.010*"работать" + 0.009*"говорить" + 0.009*"работа" + 0.008*"знать" + 0.008*"мочь" + 0.007*"просто" + 0.007*"смена" + 0.007*"день" + 0.007*"вобщий"
Тема 1: 0.037*"масло" + 0.021*"феникс" + 0.018*"цинтрийский" + 0.018*"зачарованый" + 0.016*"маслостанция" + 0.012*"понимать" + 0.011*"работать" + 0.010*"дон" + 0.010*"просто" + 0.007*"работа"
Тема 2: 0.036*"сказать" + 0.016*"скатя" + 0.010*"день" + 0.010*"мур" + 0.009*"праздник" + 0.007*"друг" + 0.007*"мёд" + 0.006*"рождение" + 0.005*"преми" + 0.005*"год"
Тема 3: 0.015*"скай" + 0.004*"гном" + 0.004*"опыт" + 0.004*"скайть" + 0.004*"какой" + 0.004*"вместо" + 0.004*"хотеться" + 0.004*"солнце" + 0.003*"начальство" + 0.002*"обучение"


## Визуализация

In [127]:
vis_data = gensimvis.prepare(best_lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)

# Задача 1. Отмечаем тематики по каждому тексту и сохраняем в json файл

In [133]:
def generate_topic_descriptions(model, num_words=5):
    topic_descriptions = {}
    for idx, topic in model.print_topics(-1):
        words = [word.split('*')[1].replace('"', '').strip() for word in topic.split(' + ')]
        topic_descriptions[idx] = ' '.join(words[:num_words])
    return topic_descriptions

topic_descriptions = generate_topic_descriptions(best_lda_model)

In [134]:
def get_topic_name(topic_distribution, topic_descriptions):
    topic_distribution = np.array([prob for _, prob in topic_distribution])
    topic_idx = topic_distribution.argmax()
    return topic_descriptions[topic_idx]

data['tematick'] = [get_topic_name(best_lda_model[doc], topic_descriptions) for doc in corpus]

In [135]:
data

,id,quote,tokens,tematick
0,0,нас среди ночи районе пятьдесят пять часов упа...,"[среди, ночь, район, пятьдесят, пять, час, упа...",человек работать говорить работа знать
1,1,програмы повышения квалификаци через гильдию ц...,"[програм, повышение, квалификаци, гильдия, цин...",масло феникс цинтрийский зачарованый маслостанция
2,2,мур новиград город контрастов да но цинтрийски...,"[мур, новиград, город, контраст, цинтрийский, ...",масло феникс цинтрийский зачарованый маслостанция
3,3,типа вот уже полгода как мы ждем установки эти...,"[тип, полгода, ждать, установка, этот, фильтр,...",масло феникс цинтрийский зачарованый маслостанция
4,4,теперь всьё так просто раньше бумажки летали т...,"[всьё, просто, ранний, бумажка, летать, терять...",масло феникс цинтрийский зачарованый маслостанция
...,...,...,...,...
954,954,го рноде не знаю них нет тут ночных смен помое...,"[го, рнода, знать, ночной, смена, помоему, два...",человек работать говорить работа знать
955,955,праздники нас както так ну были были но вот ка...,"[праздник, организовать, сначала, вроде, восто...",масло феникс цинтрийский зачарованый маслостанция
956,956,женском колективъ всё равно склок не избежать,"[женский, колективъ, равно, склока, избежать]",скай гном опыт скайть какой
957,957,потом ещё нехвата нас очень большая нас стоит ...,"[ещё, нехват, больший, стоить, наш, помещение,...",человек работать говорить работа знать


In [137]:
result_df = data[['id', 'quote', 'tematick']]
result_df.to_json('result.json', orient='records', force_ascii=False)

# Задача 2. Функция для получения тематик по новым текстовым сообщениям

## Функция определения тематики текстового сообщения

In [140]:
def predict_topic(text, model, dictionary, preprocess_text_func, topic_descriptions):
  text = to_lowercase(text)
  text = replace_eng_with_rus(text)
  text = replace_num_with_letter_in_words(text)
  text = replace_emoji_to_rus(text)
  text = replace_abbreviations(text)
  text = replace_numbers_with_words(text)
  text = clean_text(text)
  tokens = preprocess_text(text)
  bow = dictionary.doc2bow(tokens)
  topic_distribution = model[bow]
  return get_topic_name(topic_distribution, topic_descriptions)

## Пример

In [141]:
new_text = "В древней легенде говорится, что цинтрийские масла феникса обладают магическими свойствами, способными восстановить даже самые серьезные раны и вернуть к жизни давно ушедших."
predicted_topic = predict_topic(new_text, best_lda_model, dictionary, preprocess_text, topic_descriptions)
print(f"Предсказанная тема для нового текста: {predicted_topic}")

Предсказанная тема для нового текста: масло феникс цинтрийский зачарованый маслостанция
